In [36]:
def precision_go(A,P):
    M=len(set.intersection(set(A), set(P)))
    N=len(set(P))
    if M == 0:
        return 0.0
    return float(M)/float(N)

def recall_go(A,P):
    M=len(set.intersection(set(A), set(P)))
    N=len(set(A))
    if M == 0:
        return 0.0
    return float(M)/float(N)

def score_prediction_go(A,P):
    return precision_go(A,P), recall_go(A,P)
def f_measure_go(pre, rec):
    if pre+rec>0:
        return 2.0*(pre*rec)/(pre+rec)
    else:
        return 0.0
def score_GO(A,P):
    r,c =score_prediction_go(A,P)
    return r, c, f_measure_go(r,c) 
     

In [1]:
from pickle import load, dump

In [3]:
d=load(open("Data/MetaGO_Benchmark.p", 'rb'))

In [4]:
print(len(d))

1037


In [37]:
import requests, sys, json
from pickle import load, dump


GO_link = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/"
#criteria = "/ancestors?relations=is_a%2Cpart_of%2Coccurs_in%2Cregulates"
criteria = "/ancestors?relations=is_a"


# fetch the parents for a GO term
def fetchGoParents(id):
    _,id=id.split(':')
    GOId="GO%3A"+id

    requestURL =GO_link+GOId+criteria

    r = requests.get(requestURL, headers={"Accept": "application/json"})

    if not r.ok:
        return False
    responseBody = r.text
    data=json.loads(responseBody)
    #print data
    if data["results"]==[]:
        return ["GO:"+id]
    isObsolete=data["results"][0]["isObsolete"]
    if isObsolete:
        print (isObsolete)
        return ["GO:"+id]

    parents=data["results"][0]["ancestors"]
    if parents!=[]:
        P=[str(go) for go in parents]

    return P

#iteratively form a list of parent propagating through hierarchy
def dfs_collection(id):
    visited={id:1}
    Ids={id}
    #print id
    while Ids!=set():
        nextid=Ids.pop()

        if visited[nextid]==0:
            if Ids.__contains__(nextid):
                Ids.remove(nextid)
            continue

        Gos=fetchGoParents(nextid)
        if Gos.__contains__(nextid):
            Gos.remove(nextid)
        Ids=set(list(Ids)+Gos)
        #print nextid, Ids

        visited[nextid]=0
        for x in  Gos:
            if x in visited:
                continue
            visited.update({x:1})
        #print visited

    return visited.keys()

In [38]:
import numpy as np

In [4]:
actualData={}
print('Loading MetaGO Data')
fileBP = "MetaGO_Data/test_BP.is_a"
fileMF = "MetaGO_Data/test_MF.is_a"
fileCC = "MetaGO_Data/test_CC.is_a"

proteinNameBP = np.loadtxt(fileBP, dtype=np.str, delimiter='	', usecols=[0])
goTermsBP = np.loadtxt(fileBP, dtype=np.str, delimiter='	', usecols=[-1])
proteinNameMF = np.loadtxt(fileMF, dtype=np.str, delimiter='	', usecols=[0])
goTermsMF = np.loadtxt(fileMF, dtype=np.str, delimiter='	', usecols=[-1])
proteinNameCC = np.loadtxt(fileCC, dtype=np.str, delimiter='	', usecols=[0])
goTermsCC = np.loadtxt(fileCC, dtype=np.str, delimiter='	', usecols=[-1])

goTermsBP=np.core.defchararray.rsplit(goTermsBP, sep=',')
goTermsMF=np.core.defchararray.rsplit(goTermsMF, sep=',')
goTermsCC=np.core.defchararray.rsplit(goTermsCC, sep=',')

# component wise addition to the dictionary.
for x in range(0,len(proteinNameBP)):
    key = proteinNameBP[x]
    if key not in actualData:
        actualData[key]=[]
    actualData[key].append(goTermsBP[x])
for x in range(0,len(proteinNameMF)):
    key = proteinNameMF[x]
    if key not in actualData:
        actualData[key]=[]
    actualData[key].append(goTermsMF[x])
for x in range(0,len(proteinNameCC)):
    key = proteinNameCC[x]
    if key not in actualData:
        actualData[key]=[]
    actualData[key].append(goTermsCC[x])
formated_truth={}
for x in actualData:
    bp=actualData[x][0]
    mf=actualData[x][1]
    cc=actualData[x][2]
    formated_truth.update({x:{"P":bp, "F":mf, "C":cc}})
print(len(formated_truth))

Loading MetaGO Data


In [35]:
def parsePredictions(file = "GrAPFI_CN.pred",cutt_off=0.5):
    print('Parsing predicted set')
    
    data={}
    GrAPFI_data=[]

    txt = np.loadtxt(file, dtype=np.str, delimiter=' ', usecols=[0])
    for x in txt:
        x=x.split("\t")
        GrAPFI_data.append(x)

    # add data to dictionary
    for x in GrAPFI_data:
        if len(x)==1:
            continue
        key = x[0]
        if key not in data:
            data[key]={"F":[], "C":[], "P":[]}
        if float(x[-1])>=cutt_off:
            data[key][x[2]].append("GO:"+x[1])
    return data

In [39]:
anchestorList={}
def ancestorsViaDAG(predictedNew):
    predictedNewWithAncestor=[]

    for i in predictedNew:
        if i in anchestorList:
            result=anchestorList[i]
        else:
            predictedNewWithAncestor.append(i)
            result=dfs_collection(i)
            anchestorList.update({i:result})
        for val in result:
            predictedNewWithAncestor.append(val)
            

    newset=set(predictedNewWithAncestor)
    predictedNewWithAncestor=list(newset)

    return predictedNewWithAncestor


In [100]:

data=parsePredictions(file ="GrAPFI_AA.pred", cutt_off=0.5)
    

Parsing predicted set


In [101]:
precision_p=[]
precision_f=[]
precision_c=[]
recall_p=[]
recall_f=[]
recall_c=[]
f_p=[]
f_f=[]
f_c=[]

proteins=[]
for p in formated_truth:
    if p in data:
        print(p)
        proteins.append(p)
        bp_truth=formated_truth[p]["P"]
        mf_truth=formated_truth[p]["F"]
        cc_truth=formated_truth[p]["C"]
        
        bp_pred=ancestorsViaDAG(data[p]["P"])
        mf_pred=ancestorsViaDAG(data[p]["F"])
        cc_pred=ancestorsViaDAG(data[p]["C"])
        # Bp
        i,j,k=score_GO(bp_truth, bp_pred)
        precision_p.append(i)
        recall_p.append(j)
        f_p.append(k)
        #MF
        i,j,k=score_GO(mf_truth, mf_pred)
        precision_f.append(i)
        recall_f.append(j)
        f_f.append(k)
        #print(mf_truth, mf_pred)
        #print(i, j, k)
        #CC
        i,j,k=score_GO(cc_truth, cc_pred)
        precision_c.append(i)
        recall_c.append(j)
        f_c.append(k)
        
        
        

O00625
Q8TBF4


In [102]:
sum(precision_p)/len(precision_p), sum(recall_p)/len(recall_p), sum(f_p)/len(f_p)

(0.0, 0.0, 0.0)

In [103]:
sum(precision_f)/len(precision_f),sum(recall_f)/len(recall_f),sum(f_f)/len(f_f)

(0.2857142857142857, 0.5, 0.36363636363636365)

In [104]:
sum(precision_c)/len(precision_c),sum(recall_c)/len(recall_c),sum(f_c)/len(f_c)

(0.16666666666666666, 0.03571428571428571, 0.058823529411764705)

In [106]:
GOA={}
for x in anchestorList:
    GOA.update({x:list(anchestorList[x])})


In [108]:
GOA

{'GO:0005634': ['GO:0005634',
  'GO:0043229',
  'GO:0043231',
  'GO:0110165',
  'GO:0043226',
  'GO:0005575',
  'GO:0005622',
  'GO:0043227'],
 'GO:0006508': ['GO:0006508',
  'GO:0043170',
  'GO:0019538',
  'GO:0008150',
  'GO:0006807',
  'GO:0008152',
  'GO:1901564',
  'GO:0044238',
  'GO:0071704'],
 'GO:0004197': ['GO:0004197',
  'GO:0140096',
  'GO:0003824',
  'GO:0004175',
  'GO:0008234',
  'GO:0008233',
  'GO:0003674',
  'GO:0016787'],
 'GO:0005764': ['GO:0005764',
  'GO:0043229',
  'GO:0043231',
  'GO:0110165',
  'GO:0043226',
  'GO:0005575',
  'GO:0005773',
  'GO:0000323',
  'GO:0005622',
  'GO:0043227'],
 'GO:0005576': ['GO:0005576', 'GO:0110165', 'GO:0005575'],
 'GO:0046597': ['GO:0046597',
  'GO:0065007',
  'GO:0008150',
  'GO:0043903',
  'GO:0046596',
  'GO:0050792',
  'GO:0048525',
  'GO:0052372',
  'GO:0048519',
  'GO:0050789',
  'GO:1903900',
  'GO:1903901'],
 'GO:0009615': ['GO:0009615',
  'GO:0009607',
  'GO:0008150',
  'GO:0043207',
  'GO:0044419',
  'GO:0009605',
  'G

In [107]:
dump(GOA, open('GO_ancestorList.p', 'wb'))

In [85]:
def f1_max(p, r):
    print(2.0*(p*r)/(p+r))

In [87]:
f1_max(0.26, 0.20), f1_max(0.21, 0.26), f1_max(0.32, 0.30)

0.22608695652173913
0.23234042553191492
0.3096774193548387


(None, None, None)

In [88]:
f1_max(0.26, 0.19), f1_max(0.21, 0.27), f1_max(0.32, 0.30)

0.21955555555555556
0.23625000000000002
0.3096774193548387


(None, None, None)

In [89]:
f1_max(0.25, 0.20), f1_max(0.21, 0.26), f1_max(0.31, 0.29)

0.22222222222222224
0.23234042553191492
0.29966666666666664


(None, None, None)

In [90]:
f1_max(0.26, 0.20), f1_max(0.21, 0.26), f1_max(0.33, 0.29)

0.22608695652173913
0.23234042553191492
0.30870967741935484


(None, None, None)

In [92]:
f1_max(0.26, 0.19), f1_max(0.21, 0.26), f1_max(0.33, 0.30)

0.21955555555555556
0.23234042553191492
0.3142857142857143


(None, None, None)

In [99]:
f1_max(0.28, 0.21), f1_max(0.23, 0.28), f1_max(0.33, 0.31)

0.24000000000000002
0.2525490196078432
0.3196875


(None, None, None)